<a href="https://colab.research.google.com/github/Sindhusri2005/FMML-LABS-05/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?Yes, averaging the validation accuracy across multiple splits (as in **k-fold cross-validation**) generally provides more consistent and reliable results compared to using a single train-validation split. Here's why:

---

### **1. Reduces Variance in Results**
- **Single Split Issues:** A single train-validation split may lead to results that are heavily influenced by the specific split of data, especially if the dataset is small or imbalanced.
- **Multiple Splits:** Averaging over multiple splits ensures that the performance metric accounts for variations in data, leading to a more stable estimate.

---

### **2. Uses All Data for Training and Validation**
In k-fold cross-validation:
- Each fold serves as a validation set once, and the rest of the data is used for training.
- This ensures that every data point is tested, giving a more comprehensive view of model performance.

**Example:**  
With 5-fold cross-validation, every data point contributes to both training and validation, leading to a more robust measure of accuracy.

---

### **3. Guards Against Overfitting**
- **Single Split Risk:** A high validation accuracy on one split might not generalize to unseen data, especially if the split is not representative.
- **Multiple Splits:** Averaging validation accuracy across multiple folds reduces the likelihood of overfitting to specific splits, making the results more generalizable.

---

### **4. Accounts for Dataset Characteristics**
If your dataset has:
- **Imbalanced Classes:** Stratified k-fold cross-validation ensures class proportions are maintained in each fold, providing a balanced evaluation.
- **Heterogeneity:** Multiple splits help evaluate performance on different subsets of the data, ensuring the model performs well across diverse scenarios.

---

### **5. Mitigates the "Lucky Split" Effect**
Sometimes, a single train-validation split might accidentally favor the model (e.g., by having easier examples in the validation set). Cross-validation averages out these effects, giving a more unbiased performance estimate.

---

### **6. Trade-offs of Multiple Splits**
- **Increased Computation Time:** Training the model multiple times (once per fold) increases computational cost.
- **Complexity in Interpretation:** Averaged results may mask variability between folds, so it's also useful to examine the standard deviation or range of accuracies across folds.

---

### **When Averaging Validation Accuracy is Beneficial**
- **Small Datasets:** Cross-validation helps ensure all data is utilized effectively for training and validation.
- **High Variance Results:** If single-split validation results vary widely with different splits, averaging over multiple splits provides stability.
- **Model Comparison:** Averaged validation accuracy is more reliable for comparing different models or hyperparameters.

---

### **Best Practices**
1. Use **stratified k-fold cross-validation** for imbalanced datasets to ensure class distribution in each fold is representative of the dataset.
2. Report both the **mean** and **standard deviation** of validation accuracy to assess performance consistency.
3. For very large datasets, consider **holdout validation** or **repeated cross-validation** as alternatives to save computational time.

Would you like a practical example or code for implementing cross-validation?



2. Does it give more accurate estimate of test accuracy?
Could you clarify what "it" refers to in your question? Are you asking about a specific method, statistical technique, or a model used to estimate test accuracy? For instance:

- **Cross-validation** for model evaluation?
- **A/B testing** for real-world performance?
- **Specific machine learning models** like Random Forest, Logistic Regression, etc.?

Let me know so I can provide the most accurate answer!
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
The number of iterations plays a critical role in improving the reliability and accuracy of an estimate, especially in contexts like **cross-validation**, **bootstrapping**, or other repeated experiments. Here’s how the number of iterations affects the estimate:

### 1. **Effect of Number of Iterations**:
   - **Higher Iterations**: Increasing the number of iterations generally reduces the variance of the estimate and gives a more stable and accurate approximation of test accuracy.
   - **Lower Iterations**: With fewer iterations, estimates may be more variable and less representative of the true accuracy due to insufficient sampling of data splits or test conditions.

### 2. **Diminishing Returns**:
   - While more iterations improve the estimate, the improvement is not linear. Beyond a certain point, increasing iterations further provides only marginal gains in accuracy or stability.
   - For instance, in **k-fold cross-validation**, repeating the process 10–20 times often gives a stable estimate, and further repetitions may not significantly alter the result.

### 3. **Trade-offs**:
   - **Computational Cost**: More iterations mean more computations, which can be time-consuming for large datasets or complex models.
   - **Bias-Variance Trade-off in Estimation**: The estimate becomes more precise with more iterations, as variance decreases. However, the bias of the method (e.g., inherent limitations of k-fold cross-validation) doesn’t change with more iterations.

### 4. **Better Estimate with Higher Iterations?**
   - **Yes, but only to a point.** For example:
     - Repeating cross-validation (e.g., stratified k-fold cross-validation repeated 10–20 times) provides a more robust estimate than a single pass.
     - Bootstrapping with a higher number of samples reduces variability in accuracy estimates.
   - Beyond a certain number of iterations, computational resources may be better spent on refining other aspects of the model or data preprocessing.

### Recommendation:
- Use enough iterations to ensure a stable estimate but monitor for diminishing returns.
- A typical choice might be **10 iterations for cross-validation or bootstrapping**, but this can vary depending on dataset size and computational resources.

Would you like to dive deeper into a specific scenario or method?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Increasing the number of iterations may help a model learn better from the available data, but it is not a universal solution for dealing with a very small training or validation dataset. Here are some key considerations:

### 1. **Small Training Dataset**
   - **Impact of Small Training Data**: A very small training dataset may not represent the overall data distribution, leading to underfitting or overfitting. Increasing iterations might amplify overfitting, where the model memorizes the training data but fails to generalize to new examples.
   - **Effect of Increased Iterations**: More iterations can help the model converge to a better solution when there is sufficient variability in the data. However, with very limited data, the model may simply over-optimize for the small dataset and not perform well on unseen data.

   **Better Alternatives**:
   - **Data Augmentation**: Generate more diverse training examples by transforming the existing data (e.g., flipping, rotating, adding noise).
   - **Pretrained Models**: Use transfer learning to leverage knowledge from models trained on larger datasets.
   - **Regularization**: Techniques like dropout or weight regularization can help prevent overfitting.

### 2. **Small Validation Dataset**
   - **Impact of Small Validation Data**: A small validation set may not provide reliable feedback about model performance, as it may not capture the full variability of the data.
   - **Effect of Increased Iterations**: Increasing iterations does not directly address issues with validation data. The evaluation metrics derived from a small validation dataset may not be robust or meaningful.

   **Better Alternatives**:
   - **Cross-Validation**: Use techniques like k-fold cross-validation to maximize the use of the available data for training and validation.
   - **Expand Dataset**: If possible, gather more data or merge similar datasets to create a more reliable validation set.

### Summary
While increasing iterations might help optimize the model's performance for a small dataset, it is not a replacement for having sufficient and diverse training and validation data. Instead, focus on strategies like data augmentation, transfer learning, regularization, and cross-validation to mitigate the issues caused by a small dataset.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.